In [ ]:
import numpy as np
import pandas as pd
import chaospy as cp
import sobol_seq
from string import ascii_lowercase
import numba
import matplotlib.pyplot as plt

k = 6 # number of parameters inquired

# It is assumed the sm represents the dataframe on which we will be operating
a2 = [0, 0, 6.52, 6.52, 6.52, 6.52]
b3 = [6.52, 6.52, 6.52, 6.52, 6.52, 6.52]

@numba.jit
def A1(sm):
    dummyA1 = pd.DataFrame()
    for j in range(0,k):
        dummyA1[j] = np.prod(sm[sm.columns[0:j+1]], axis = 1)*(-1)**(j+1)
    return dummyA1.sum(axis=1)
        
def A2(sm):
    dummyA2 = pd.DataFrame()
    for j in range(0,k):
        dummyA2[j] = (abs(4*sm[sm.columns[j]]-2)+a2[j])/(1+a2[j])
    return (dummyA2.product(axis=1))
        
def B1(sm):
    dummyB1 = pd.DataFrame()
    for j in range(0,k):
        dummyB1[j] = (k-sm[sm.columns[j]])/(k-0.5)
    return dummyB1.product(axis=1)
        
def B2(sm):
    dummyB2 = pd.DataFrame()
    for j in range(0,k):
        dummyB2[j] = (sm[sm.columns[j]])**(1/k)
    return dummyB2.product(axis=1)*((1+1/k)**k)
        
def B3(sm):
    dummyB3 = pd.DataFrame()
    for j in range(0,k):
        dummyB3[j] = (abs(4*sm[sm.columns[j]]-2)+b3[j])/(1+b3[j])
    return (dummyB3.product(axis=1))
        
def C1(sm):
    dummyC1 = pd.DataFrame()
    for j in range(0,k):
        dummyC1[j] = pd.Series(abs(4*sm[sm.columns[j]]-2))
    return (dummyC1.product(axis=1))
        
def C2(sm):
    return (sm.product(axis=1)*2**k)

functions = [A1, A2, B1, B2, B3, C1, C2]

#AEA1

def Ek2(k):
    return((1/6)*(1-(1/3)**k)+(4/15)*((-1)**(k+1)*(1/2)**k+(1/3)**k))

def V(k):
    return((1/10)*(1/3)**k+(1/18)-(1/9)*(1/2)**(2*k)+(-1)**(k+1)*(2/45)*(1/2)**k)

for j in range(0, k):
    def Ej(j):
        return((1/6)*(1-(1/3)**(j))+(4/15)*((-1)**(j+1)*(1/2)**(j)+(1/3)**(j)))
              
    def T1(j):
        return((1/2)*((1/3)**(j-1)*(1-(1/3)**(k-j))))

    def T2(j):
        return((1/2)*((1/3)**j-(1/3)**k))
    
    def T3(j):
        return((3/5)*(4*(1/3)**(k+1)+(-1)**(j+k+1)*(1/2)**(k-j-2)*(1/3)**(j+1)))
    
    def T4(j):
        return((1/5)*((-1)**(j+2)*(1/3)*(1/2)**(j-2)-4*(1/3)**(j+1)))
    
    def T5(j):
        return((1/5)*((-1)**(k+1)*(1/3)*(1/2)**(k-2)+(-1)**(k+j)*(1/3)**(j+1)*(1/2)**(k-j-2)))
    
def A1ST(j):
    return((Ek2(k)-Ej(j)-(1/4)*(T1(j)-2*T2(j)+T3(j))-T4(j)-T5(j))/V(k))

#ÁEA2

def VjA2(j):
    return((1/3)/(1+a2[j])**2)

def VA2(j):
    productA2 = []
    for j in range(0, k):
        productA2.append(1+VjA2(j))
    return np.product(productA2)-1

def VnA2(j):
    return((VA2(j)+1)/(1+VjA2(j)))

def VTjA2(j):
    return VjA2(j)*(VnA2(j))

def A2ST(j):
    return(VTjA2(j)/VA2(j))

#AEB1

def p(j):
    return(12*(k-0.5)**2)
        
def B1ST(j):
    return((p(j)+1)**k/((p(j)+1)*((p(j)+1)**k-p(j)**k)))

#AEB2

def B2ST(j):
    return((k+1)**(2*k-2)/(((k+1)**(2*k)-(k**2+2*k)**k)))

#AEB3

def VjB3(j):
    return((1/3)/(1+b3[j])**2)

def VB3(j):
    productB3 = []
    for j in range(0, k):
        productB3.append(1+VjB3(j))
    return np.product(productB3)-1

def VnB3(j):
    return((VB3(j)+1)/(1+VjB3(j)))

def VTjB3(j):
    return VjB3(j)*(VnB3(j))

def B3ST(j):
    return(VTjB3(j)/VB3(j))

#AEC1

def C1ST(j):
    return 4**(k-1)/(4**k-3**k)

#AEC2
        
def C2ST(j):
    return 4**(k-1)/(4**k-3**k)

def create_dict(key, values):
    return dict(zip(key, values))

analyticalValues = [A1ST, A2ST, B1ST, B2ST, B3ST, C1ST, C2ST]

AE = []
AE_names = []
for w in analyticalValues:
    for j in range (0,k):
        AE.append(w(j))
        AE_names.append('AE' + str(w.__name__) + str(j+1))
AE_dic = create_dict(AE_names, AE)

AE = []
AE_names = []
for w in analyticalValues:
    for j in range (0,k):
        AE.append(w(j))
        AE_names.append('AE' + str(w.__name__) + str(j+1))
AE_dic = create_dict(AE_names, AE)

letters = []

for l in ascii_lowercase:
    letters.append(l)
    
p = 11
n = [2,3,4,6]
run = 50

p_sample = []
p_sample_name = []

n_matrix = []
n_matrix_name = []
df = pd.DataFrame(sobol_seq.i4_sobol_generate(k, 12*len(n)*run*(2**p-1)))
df = df.sample(frac=1).reset_index(drop=True)

In [ ]:
qamples = []
for s in range (p):
    qamples.append(df.iloc[12*run*len(n)*(-1+2**s):12*run*len(n)*(-1+2**(s+1))].reset_index(drop=True))
    
    samples = []
    for idx, n1 in enumerate(n):
        samples.append(qamples[s].iloc[int(idx*(len(qamples[s].index)/len(n))):int((idx+1)*(len(qamples[s].\
        index)/len(n)))].reset_index(drop=True))
        LambMAE_mean = []
    
        run_samples = []
        for r in range (run):
            run_samples.append(samples[idx].iloc[int(r*(len(samples[idx].index)/run)):int((r+1)*(len(samples[idx].\
            index)/run))].reset_index(drop=True))
        
            sample_Matrices = []
            for m in range (0, n1):
                sample_Matrices.append(run_samples[r].iloc[int(m*(len(run_samples[r].index)/n1)):int((m+1)*\
                (len(run_samples[r].index)/n1))].reset_index(drop=True))
            sample_Matrices_dic = create_dict(letters, sample_Matrices)
    
            mixed_Matrices = []
            mm_names = []
            for sm in range (0,len(sample_Matrices)):
                for sm1 in range (0,len(sample_Matrices)):
                    if sm == sm1:
                        continue
                    else:
                        for c in sample_Matrices[sm]:
                            mixed_Matrices.append(sample_Matrices[sm].copy())
                            mixed_Matrices[len(mixed_Matrices)-1][c]=sample_Matrices[sm1].copy()[c]
                            mm_names.append(str(letters[sm] + letters[sm1] + str(c+1)))
            mixed_Matrices_dic = create_dict(mm_names, mixed_Matrices)
            
            names = []
            names1 = []
            values = []
            values1 = []
            for f in functions:
                for ss, zs in sample_Matrices_dic.items():
                    names.append(f.__name__+str(ss))
                    values.append(f(zs))
    
                for sq, zq in mixed_Matrices_dic.items():
                    names1.append(f.__name__+str(sq))
                    values1.append(f(zq))
            f_SM_dic = create_dict(names, values)
            f_MM_dic = create_dict(names1, values1)
            
            Lamb=[]
            LambName = []
            for f in functions:
                for j in range(1,k+1):
                    difference = []
                    for ck, z in f_SM_dic.items():
                        if ck[0:2]==f.__name__:
                            validkeys = []
                            for fk1 in f_MM_dic.keys():
                                if fk1[0:3]==ck[0:3] and fk1[4]==str(j):
                                    validkeys.append(fk1)
                            z1 = dict(filter(lambda i:i[0] in validkeys, f_MM_dic.items()))
                            difference.append(((n1-1)/n1**2)*(((z-(1/(n1-1))*sum(z1.values()))**2).mean())/z.var())
                    Lamb.append(sum(difference))
                    LambName.append('Lamboni'+f.__name__+'ST'+str(j))
            Lamb_dic = create_dict(LambName, Lamb)
    
            LambMAEs = []
            LambMAENames = []
            for ae, av in AE_dic.items():
                for Lk, Lv in Lamb_dic.items():
                    if ae[-5:]==Lk[-5:]:
                        LambMAEs.append(abs(Lv-av))
                        LambMAENames.append('LamboniMAE'+ str(ae[2:4]) + str(ae[-1]))
            LambMAEs_dic = create_dict(LambMAENames, LambMAEs)
    
            LambMAE = []
            LambMAE_name = []
            for f in functions:
                validkeys2 = []
                for Lmk, Lmv in LambMAEs_dic.items():
                    if Lmk[-3:-1]==f.__name__:
                        validkeys2.append(Lmk)
                z2 = dict(filter(lambda i:i[0] in validkeys2, LambMAEs_dic.items()))
                LambMAE.append(sum(z2.values())/len(z2))
                LambMAE_name.append('LamboniMAE'+f.__name__)
            LambMAE_dic = create_dict(LambMAE_name, LambMAE)
            LambMAE_mean.append(LambMAE_dic)
        LambMAE_mean_dic = {Lmk1:[LambMAE_mean[Lmv1][Lmk1] for Lmv1 in range(len(LambMAE_mean))] for Lmk1 in LambMAE_mean[0].keys()}
        for Lmk1,Lmv1 in LambMAE_mean_dic.items():
            LambMAE_mean_dic[Lmk1] = sum(LambMAE_mean_dic[Lmk1])/len(LambMAE_mean_dic[Lmk1])
        n_matrix.append(LambMAE_mean_dic)
        n_matrix_name.append(str(n1)+'matrices')
    n_matrix_dic = create_dict(n_matrix_name, n_matrix)
    p_sample.append(n_matrix_dic)
    p_sample_name.append(12*2**s)
p_sample_dic = create_dict(p_sample_name, p_sample)
Lamboniq = pd.DataFrame.from_dict({(i2,j2): p_sample_dic[i2][j2] 
                                   for i2 in p_sample_dic.keys() 
                                   for j2 in p_sample_dic[i2].keys()},
                                  orient='index').unstack(level=1).T
writer = pd.ExcelWriter('C:/Users/Samuele/Dropbox (Personal)/Research/Sensitivity Analysis/Lamboni_all.xlsx')

In [ ]:
qamples = []
for s in range (p):
    qamples.append(df.iloc[12*run*len(n)*(-1+2**s):12*run*len(n)*(-1+2**(s+1))].reset_index(drop=True))
    
    samples = []
    for idx, n1 in enumerate(n):
        samples.append(qamples[s].iloc[int(idx*(len(qamples[s].index)/len(n))):int((idx+1)*(len(qamples[s].\
        index)/len(n)))].reset_index(drop=True))
        CheckMAE_mean = []
    
        run_samples = []
        for r in range (run):
            run_samples.append(samples[idx].iloc[int(r*(len(samples[idx].index)/run)):int((r+1)*(len(samples[idx].\
            index)/run))].reset_index(drop=True))
        
            sample_Matrices = []
            for m in range (0, n1):
                sample_Matrices.append(run_samples[r].iloc[int(m*(len(run_samples[r].index)/n1)):int((m+1)*\
                (len(run_samples[r].index)/n1))].reset_index(drop=True))
                
            sample_Matrices_dic = create_dict(letters, sample_Matrices)
        
            mixed_Matrices = []
            mm_names = []
            for sm in range (0,len(sample_Matrices)):
                for sm1 in range (0,len(sample_Matrices)):
                    if sm == sm1:
                        continue
                    else:
                        for c in sample_Matrices[sm]:
                            mixed_Matrices.append(sample_Matrices[sm].copy())
                            mixed_Matrices[len(mixed_Matrices)-1][c]=sample_Matrices[sm1].copy()[c]
                            mm_names.append(str(letters[sm] + letters[sm1] + str(c+1)))
            mixed_Matrices_dic = create_dict(mm_names, mixed_Matrices)

            matrices_dic = {**sample_Matrices_dic, **mixed_Matrices_dic}
            
            names = []
            names1 = []
            values = []
            values1 = []
            names2 = []
            values2 = []
            for f in functions:
                for ss, zs in sample_Matrices_dic.items():
                    names.append(f.__name__+str(ss))
                    values.append(f(zs))
    
                for sq, zq in mixed_Matrices_dic.items():
                    names1.append(f.__name__+str(sq))
                    values1.append(f(zq))
                    
                for sM, zM in matrices_dic.items():
                    names2.append(f.__name__+str(sM))
                    values2.append(f(zM))
                    
            f_SM_dic = create_dict(names, values)
            f_MM_dic = create_dict(names1, values1)
            f_matrices_dic = create_dict(names2, values2)
            
            Check=[]
            CheckName = []
            for f in functions:
                for j in range(1,k+1):
                    difference = []
                    for mk, mz in f_matrices_dic.items():
                        if mk[0:2]==f.__name__:
                            validkeys = []
                            for fk1 in f_MM_dic.keys():
                                if len(mk)==3: 
                                    if fk1[0:3]==mk[0:3] and fk1[-1]==str(j):
                                        validkeys.append(fk1)
                                else: 
                                    if fk1[0:3]==mk[0:3] and fk1[-1]==mk[-1] and fk1!=mk and fk1>=mk and fk1[-1]==str(j):
                                        validkeys.append(fk1)
                            z1 = dict(filter(lambda i:i[0] in validkeys, f_MM_dic.items()))
                            for zk, zv in z1.items():
                                difference.append(0.5*(((mz-zv)**2).mean())/mz.var())
                    Check.append(sum(difference)/len(difference))
                    CheckName.append('Jansen'+ str(f.__name__) +'ST'+str(j))
            Check_dic = create_dict(CheckName, Check)
                    
            CheckMAEs = []
            CheckMAENames = []
            for ae, av in AE_dic.items():
                for Lk, Lv in Check_dic.items():
                    if ae[-5:]==Lk[-5:]:
                        CheckMAEs.append(abs(Lv-av))
                        CheckMAENames.append('CheckMAE'+ str(ae[2:4]) + str(ae[-1]))
            CheckMAEs_dic = create_dict(CheckMAENames, CheckMAEs)
    
            CheckMAE = []
            CheckMAE_name = []
            for f in functions:
                validkeys2 = []
                for Lmk, Lmv in CheckMAEs_dic.items():
                    if Lmk[-3:-1]==f.__name__:
                        validkeys2.append(Lmk)
                z2 = dict(filter(lambda i:i[0] in validkeys2, CheckMAEs_dic.items()))
                CheckMAE.append(sum(z2.values())/len(z2))
                CheckMAE_name.append('CheckMAE'+f.__name__)
            CheckMAE_dic = create_dict(CheckMAE_name, CheckMAE)
            CheckMAE_mean.append(CheckMAE_dic)
        CheckMAE_mean_dic = {Lmk1:[CheckMAE_mean[Lmv1][Lmk1] for Lmv1 in range(len(CheckMAE_mean))] for Lmk1 in CheckMAE_mean[0].keys()}
        for Lmk1,Lmv1 in CheckMAE_mean_dic.items():
            CheckMAE_mean_dic[Lmk1] = sum(CheckMAE_mean_dic[Lmk1])/len(CheckMAE_mean_dic[Lmk1])
        n_matrix.append(CheckMAE_mean_dic)
        n_matrix_name.append(str(n1)+'matrices')
    n_matrix_dic = create_dict(n_matrix_name, n_matrix)
    p_sample.append(n_matrix_dic)
    p_sample_name.append(12*2**s)
p_sample_dic = create_dict(p_sample_name, p_sample)
Checkq = pd.DataFrame.from_dict({(i2,j2): p_sample_dic[i2][j2] 
                                   for i2 in p_sample_dic.keys() 
                                   for j2 in p_sample_dic[i2].keys()},
                                  orient='index').unstack(level=1).T
writer = pd.ExcelWriter('C:/Users/Samuele/Dropbox (Personal)/Research/Sensitivity Analysis/Check_all.xlsx')

In [ ]:
n = [2]
qamples = []
for s in range (p):
    qamples.append(df.iloc[12*run*4*(-1+2**s):12*run*4*(-1+2**(s+1))].reset_index(drop=True))
    
    samples = []
    for idx, n1 in enumerate(n):
        samples.append(qamples[s].iloc[int(idx*(len(qamples[s].index)/4)):int((idx+1)*(len(qamples[s].\
        index)/4))].reset_index(drop=True))
        CheckMAE_mean = []
    
        run_samples = []
        for r in range (run):
            run_samples.append(samples[idx].iloc[int(r*(len(samples[idx].index)/run)):int((r+1)*(len(samples[idx].\
            index)/run))].reset_index(drop=True))
        
            sample_Matrices = []
            for m in range (0, n1):
                sample_Matrices.append(run_samples[r].iloc[int(m*(len(run_samples[r].index)/n1)):int((m+1)*\
                (len(run_samples[r].index)/n1))].reset_index(drop=True))
                
            sample_Matrices_dic = create_dict(letters, sample_Matrices)
        
            mixed_Matrices = []
            mm_names = []
            for sm in range (0,len(sample_Matrices)):
                for sm1 in range (0,len(sample_Matrices)):
                    if sm == sm1:
                        continue
                    else:
                        for c in sample_Matrices[sm]:
                            mixed_Matrices.append(sample_Matrices[sm].copy())
                            mixed_Matrices[len(mixed_Matrices)-1][c]=sample_Matrices[sm1].copy()[c]
                            mm_names.append(str(letters[sm] + letters[sm1] + str(c+1)))
            mixed_Matrices_dic = create_dict(mm_names, mixed_Matrices)

            matrices_dic = {**sample_Matrices_dic, **mixed_Matrices_dic}
            
            names = []
            names1 = []
            values = []
            values1 = []
            names2 = []
            values2 = []
            for f in functions:
                for ss, zs in sample_Matrices_dic.items():
                    names.append(f.__name__+str(ss))
                    values.append(f(zs))
    
                for sq, zq in mixed_Matrices_dic.items():
                    names1.append(f.__name__+str(sq))
                    values1.append(f(zq))
                    
                for sM, zM in matrices_dic.items():
                    names2.append(f.__name__+str(sM))
                    values2.append(f(zM))
                    
            f_SM_dic = create_dict(names, values)
            f_MM_dic = create_dict(names1, values1)
            f_matrices_dic = create_dict(names2, values2)
            
            Check=[]
            CheckName = []
            for f in functions:
                for j in range(1,k+1):
                    difference = []
                    for mk, mz in f_matrices_dic.items():
                        if mk[0:2]==f.__name__:
                            validkeys = []
                            for fk1 in f_MM_dic.keys():
                                if len(mk)==3: 
                                    if mk[2] == 'a' and fk1[0:3]==mk[0:3] and fk1[-1]==str(j):
                                        validkeys.append(fk1)
                                else: 
                                    if fk1[0:3]==mk[0:3] and fk1[-1]==mk[-1] and fk1!=mk and fk1>=mk and fk1[-1]==str(j):
                                        validkeys.append(fk1)
                            z1 = dict(filter(lambda i:i[0] in validkeys, f_MM_dic.items()))
                            for zk, zv in z1.items():
                                difference.append(0.5*(((mz-zv)**2).mean())/mz.var())
                    Check.append(sum(difference)/len(difference))
                    CheckName.append('Jansen'+ str(f.__name__) +'ST'+str(j))
            Check_dic = create_dict(CheckName, Check)
                    
            CheckMAEs = []
            CheckMAENames = []
            for ae, av in AE_dic.items():
                for Lk, Lv in Check_dic.items():
                    if ae[-5:]==Lk[-5:]:
                        CheckMAEs.append(abs(Lv-av))
                        CheckMAENames.append('CheckMAE'+ str(ae[2:4]) + str(ae[-1]))
            CheckMAEs_dic = create_dict(CheckMAENames, CheckMAEs)
    
            CheckMAE = []
            CheckMAE_name = []
            for f in functions:
                validkeys2 = []
                for Lmk, Lmv in CheckMAEs_dic.items():
                    if Lmk[-3:-1]==f.__name__:
                        validkeys2.append(Lmk)
                z2 = dict(filter(lambda i:i[0] in validkeys2, CheckMAEs_dic.items()))
                CheckMAE.append(sum(z2.values())/len(z2))
                CheckMAE_name.append('CheckMAE'+f.__name__)
            CheckMAE_dic = create_dict(CheckMAE_name, CheckMAE)
            CheckMAE_mean.append(CheckMAE_dic)
        CheckMAE_mean_dic = {Lmk1:[CheckMAE_mean[Lmv1][Lmk1] for Lmv1 in range(len(CheckMAE_mean))] for Lmk1 in CheckMAE_mean[0].keys()}
        for Lmk1,Lmv1 in CheckMAE_mean_dic.items():
            CheckMAE_mean_dic[Lmk1] = sum(CheckMAE_mean_dic[Lmk1])/len(CheckMAE_mean_dic[Lmk1])
        n_matrix.append(CheckMAE_mean_dic)
        n_matrix_name.append(str(n1)+'matrices')
    n_matrix_dic = create_dict(n_matrix_name, n_matrix)
    p_sample.append(n_matrix_dic)
    p_sample_name.append(12*2**s)
p_sample_dic = create_dict(p_sample_name, p_sample)
Checkq = pd.DataFrame.from_dict({(i2,j2): p_sample_dic[i2][j2] 
                                   for i2 in p_sample_dic.keys() 
                                   for j2 in p_sample_dic[i2].keys()},
                                  orient='index').unstack(level=1).T

In [ ]:
Checkq.index = Checkq.index.droplevel(0)

In [ ]:
Lamboniq.index = Lamboniq.index.droplevel(0)

In [ ]:
Percentage_Error = (Checkq-Lamboniq)*100/Checkq

In [ ]:
Percentage_Error